# 동영상으로부터 음성데이터 추출하기

In [2]:
!sudo apt update && sudo apt install ffmpeg
!pip install -q openai-whisper pytube tiktoken
!pip install moviepy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,188 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,527 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

In [4]:
from google.colab import files

# 파일 업로드 인터페이스 호출
uploaded = files.upload()

# 업로드한 파일 이름 확인
for filename in uploaded.keys():
    print('Uploaded file "{name}" with length {length} bytes'.format(
        name=filename, length=len(uploaded[filename])))

# 파일명을 'filename' 변수에 저장
filename = next(iter(uploaded))

# 파일을 바이너리 모드로 열기 (이진 파일 처리 예제)
with open(filename, 'rb') as file:
    content = file.read()
    print(f"Read {len(content)} bytes from the file.")


Saving my_voice.mp4 to my_voice.mp4
Uploaded file "my_voice.mp4" with length 79958507 bytes
Read 79958507 bytes from the file.


In [5]:
from moviepy.editor import VideoFileClip

# Colab의 sample_data 디렉토리에 있는 동영상 파일 경로
# 예를 들어, 동영상 파일 이름이 'example_video.mp4'라고 가정
video_path = '/content/sample_data/my_voice.mp4'

# 추출된 오디오를 저장할 파일 이름 설정
audio_output_path = '/content/sample_data/extracted_audio.mp3'

# 동영상 파일 로드
video_clip = VideoFileClip(video_path)

# 오디오 추출
audio_clip = video_clip.audio
# 오디오 파일로 저장
audio_clip.write_audiofile(audio_output_path)

# 메모리 해제
audio_clip.close()
video_clip.close()

# 생성된 오디오 파일 다운로드 (필요한 경우)
from google.colab import files
files.download(audio_output_path)


  if event.key is 'enter':



MoviePy - Writing audio in /content/sample_data/extracted_audio.mp3


MoviePy - Done.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
# Whisper 모델 불러오기
import whisper

# Size =['tiny', 'base', 'small', 'medium', 'large', 'turbo']
model = whisper.load_model("small")

# 오디오 파일의 경로 (Colab의 sample_data 디렉토리에 있는 파일이라고 가정)
audio_path = '/content/sample_data/extracted_audio.mp3'

# 오디오 파일에서 음성 인식 수행
result = model.transcribe(audio_path)

# 인식된 텍스트의 첫 300자 출력
#print(result["text"][:300])

  checkpoint = torch.load(fp, map_location=device)



In [7]:
len(result["text"])

5217

In [17]:
# cnt=0
# for r in result['segments']:
#     print(f'[{r["start"]:.1f} --> {r["end"]:.1f}] {r["text"]}')
#     cnt+=1

# print()
# print(cnt)

In [10]:
!mkdir wavs

In [11]:
import os

# wavs 폴더 내 모든 파일 삭제
def delete_all_files_in_folder(folder_path):
    if not os.path.exists(folder_path):
        print(f"폴더 {folder_path}가 존재하지 않습니다.")
        return

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        try:
            if os.path.isfile(file_path):  # 파일인지 확인
                os.remove(file_path)
                print(f"삭제 완료: {file_path}")
        except Exception as e:
            print(f"파일 삭제 중 오류 발생: {file_path}, {e}")

# 사용 예시
delete_all_files_in_folder("wavs")


In [12]:
import os

# 각 segment를 더 작은 interval로 나누기
def split_segment_with_text(segment, interval=1.0):
    """segment를 일정한 간격(interval)으로 나누고, 텍스트를 해당 구간에 맞게 분리."""
    start = segment["start"]
    end = segment["end"]
    full_text = segment["text"].strip()

    sub_segments = []
    current_start = start

    # 전체 segment 길이와 텍스트 길이를 기준으로 비율 계산
    segment_duration = end - start
    text_length = len(full_text)

    if text_length > 0:  # 텍스트가 있는 경우 비율에 따라 분리
        chars_per_second = text_length / segment_duration
    else:  # 텍스트가 비어 있는 경우 빈 문자열로 채움
        chars_per_second = 0

    while current_start < end:
        current_end = min(current_start + interval, end)
        text_start_idx = int((current_start - start) * chars_per_second)
        text_end_idx = int((current_end - start) * chars_per_second)
        text_slice = full_text[text_start_idx:text_end_idx].strip()

        if text_slice:  # 텍스트가 비어있지 않은 경우에만 추가
            sub_segments.append({
                "start": current_start,
                "end": current_end,
                "text": text_slice
            })

        current_start = current_end

    return sub_segments

# 새로운 segment 리스트 생성
interval = 3.0  # 각 작은 segment의 길이 (초)
new_segments = []
for segment in result['segments']:
    new_segments.extend(split_segment_with_text(segment, interval=interval))

# 출력 폴더 생성
os.makedirs("wavs", exist_ok=True)

# 각 sub-segment를 별도의 WAV 파일로 저장
audio_index = 1  # 연속적인 파일 번호를 위한 변수
metadata_lines = []

for r in new_segments:
    start = round(r["start"], 3)
    end = round(r["end"], 3)
    output_path = f"wavs/audio{audio_index}.wav"
    os.system(f"ffmpeg -y -i {audio_path} -ss {start} -to {end} -hide_banner -loglevel error {output_path}")

    # Metadata 작성
    metadata_lines.append(f"audio{audio_index}|{r['text']}|{r['text']}")
    audio_index += 1

# metadata.txt 작성
with open("metadata.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(metadata_lines))


In [13]:
from IPython.display import Audio

Audio("wavs/audio1.wav")

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
!mkdir /content/drive/MyDrive/bbanghyong-tts-dataset
!zip -r bbanghyong.zip /content/wavs /content/metadata.txt
!cp -a /content/bbanghyong.zip /content/drive/MyDrive/bbanghyong-tts-dataset

mkdir: cannot create directory ‘/content/drive/MyDrive/bbanghyong-tts-dataset’: File exists
  adding: content/wavs/ (stored 0%)
  adding: content/wavs/audio122.wav (deflated 17%)
  adding: content/wavs/audio126.wav (deflated 15%)
  adding: content/wavs/audio216.wav (deflated 27%)
  adding: content/wavs/audio232.wav (deflated 23%)
  adding: content/wavs/audio168.wav (deflated 20%)
  adding: content/wavs/audio175.wav (deflated 27%)
  adding: content/wavs/audio228.wav (deflated 21%)
  adding: content/wavs/audio32.wav (deflated 15%)
  adding: content/wavs/audio279.wav (deflated 20%)
  adding: content/wavs/audio62.wav (deflated 21%)
  adding: content/wavs/audio128.wav (deflated 19%)
  adding: content/wavs/audio196.wav (deflated 18%)
  adding: content/wavs/audio55.wav (deflated 27%)
  adding: content/wavs/audio180.wav (deflated 19%)
  adding: content/wavs/audio173.wav (deflated 19%)
  adding: content/wavs/audio37.wav (deflated 19%)
  adding: content/wavs/audio277.wav (deflated 25%)
  adding: